# Read an plot Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def readTrajectory(file):
    ''' Read in Trajectory which was written in "Render.txt" before and return it in numpy arrays:
    Field (length_of_traj, 4, 4), Actions (length_of_traj)
    Field:
        1              - Start
        2              - Ice
        3              - Hole
        4              - Goal
        negative value - position of agent
    Actions:
        0 - Left
        1 - Down
        2 - Right
        3 - Up
        4 - Do nothing
    '''
    f = open(file, 'r')
    if f.mode == 'r':
        content = f.read()
    f.close()

    i = 0
    steps = 0
    while i < len(content):
        #print(content[i], ord(content[i]))
        if ord(content[i]) == 27:
            content = content[:i] + content[i+5] + 'x' + content[i+10:]
            steps += 1
        i += 1

    field = np.zeros(steps*4*4)
    directions = {0:'left', 1:'down', 2:'right', 3:'up'}
    actions = np.zeros(steps)
    j = 0
    k = 0
    #print(content)
    for i in range(len(content)):
        if content[i] == 'S':
            field[j] = 1
            j += 1
        elif content[i] == 'F':
            field[j] = 2
            j += 1
        elif content[i] == 'H':
            field[j] = 3
            j += 1
        elif content[i] == 'G':
            field[j] = 4
            j += 1
        elif content[i] == 'x':
            field[j-1] = -field[j-1]
        elif content[i] == '(':
            if content[i+1] == 'L':
                actions[k] = 0
                k += 1
            elif content[i+1] == 'D':
                actions[k] = 1
                k += 1
            elif content[i+1] == 'R':
                actions[k] = 2
                k += 1
            elif content[i+1] == 'U':
                actions[k] = 3
                k += 1

    field = field.reshape(steps, 4, 4)
    actions = actions.astype(int)
    actions[-1] = 4 # "do nothing"

    return field, actions

field, actions = readTrajectory('Render.txt')
steps = field.shape[0]

In [ ]:
from PIL import Image
''' Load Image of dot and arrow and transform it to
show actions in each step
'''
dot = Image.open('Data/dot.tiff')
dot = np.array(dot)
larrow = Image.open('Data/left-arrow.tiff')
larrow = np.array(larrow)
uarrow = larrow.T
rarrow = np.flip(larrow, 1)
darrow = np.flip(uarrow, 0)
arrows = np.array([larrow, darrow, rarrow, uarrow, dot])

In [ ]:
import matplotlib as mpl
''' Define colormap and normalization to render the frozen lake
appropriately
'''
color_list = [(252/255, 40/255, 40/255), #red
              (39/255, 173/255, 9/255), # green
              (232/255, 252/255, 250/255), #ice white
              (0/255, 28/255, 25/255), #dark
              (252/255, 208/255, 30/255) #golden
             ]
frozen_lake_cmap = mpl.colors.ListedColormap(color_list, name='frozen_lake_cmap')
plt.register_cmap(cmap=frozen_lake_cmap)
norm = mpl.colors.Normalize(vmin=0, vmax=4)

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.ticker import MaxNLocator
from matplotlib.lines import Line2D
''' Show animation of the Trajectory with customly made legend elements
'''

fig, (ax1,ax2) = plt.subplots(1,2)

ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))
ax2.axis('off')
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Position',
                          markerfacecolor=(252/255, 40/255, 40/255), markersize=15),
                  Line2D([0], [0], marker='o', color='w', label='Start',
                          markerfacecolor=(39/255, 173/255, 9/255), markersize=15),
                  Line2D([0], [0], marker='o', color='w', label='Ice',
                          markerfacecolor=(232/255, 252/255, 250/255), markersize=15),
                  Line2D([0], [0], marker='o', color='w', label='Hole',
                          markerfacecolor=(0/255, 28/255, 25/255), markersize=15),
                  Line2D([0], [0], marker='o', color='w', label='Goal',
                          markerfacecolor=(252/255, 208/255, 30/255), markersize=15)]
ax1.legend(handles=legend_elements, loc = 3, bbox_to_anchor=(0,1.1,1,0.2), ncol=5)

fig.suptitle('t = 0', fontsize=16)
im1 = ax1.imshow(field[0], animated=True, cmap=frozen_lake_cmap, norm=norm)
im2 = ax2.imshow(arrows[actions[0]], cmap='gray', animated=True)

i = 0
def updatefig(*args):
    global i
    fig.suptitle('t = {}'.format(i), fontsize=16)
    im1.set_array(field[i])
    im2.set_array(arrows[actions[i]])
    i += 1
    if i >= field.shape[0]:
        i = 0
    return im1,im2

anim = animation.FuncAnimation(fig, updatefig, frames=steps-1, interval=1000, blit=True, repeat_delay=5000)
plt.show()